In [1]:
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import tqdm
from qdrant_client import QdrantClient, models
from flask import Flask, request, jsonify
from werkzeug.serving import run_simple
from flask_cors import CORS

def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    text = text.replace(" ", "")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

def format_prompt(question, search_results):
    results = ""
    for i in range(5):
        results = results + search_results[i].payload['text']
    # print(results)
    return results

def get_messages(quest):
    question = quest
    search_results = client_qd.search(
        collection_name='putin',
        query_vector=get_embedding(question),
        limit=5,
        search_params={"exact": False, "hnsw_ef": 128}
    )
    return format_prompt(question, search_results)

client = OpenAI()
client_qd = QdrantClient("127.0.0.1", port=6333)


app = Flask(__name__)
CORS(app)  # 添加CORS支持

res = ""
# 定义一个路由来处理接收到的 content
@app.route('/process_content', methods=['POST'])
def process_content():
    global res
    content = request.json.get('content')
    
    # 在这里可以对接收到的 content 进行处理，比如保存到文件、执行某些操作等
    res = get_messages(content)

    return res

# 定义一个路由来获取数据
@app.route('/get_data', methods=['GET'])            
def get_data():
    # 这里是你的Python代码，用于获取数据
    global res
    data = res
    return jsonify({'data': data})


# 在Notebook中启动Flask应用
from werkzeug.serving import run_simple
run_simple('localhost', 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Apr/2024 13:04:50] "OPTIONS /process_content HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2024 13:04:54] "POST /process_content HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2024 13:22:39] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2024 13:22:39] "GET /process_content HTTP/1.1" 405 -
127.0.0.1 - - [11/Apr/2024 13:22:52] "OPTIONS /process_content HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2024 13:22:56] "POST /process_content HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2024 13:23:46] "OPTIONS /process_content HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2024 13:23:49] "POST /process_content HTTP/1.1" 200 -
